# Loan-classifier ML

## Importando bibliotecas

In [156]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

print(sklearn.__version__)

1.5.1


## Entendendo a base

In [159]:
db = pd.read_csv('../DataSets/loan.csv')
df = db.copy()

In [161]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [163]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [165]:
df['Dependents'].unique()

array(['0', '1', '2', '3+', nan], dtype=object)

A coluna *Dependents* é do tipo string, por causa do tipo de dado "+3" presente, quando deveria ser int. 

In [168]:
df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [170]:
df.shape

(614, 13)

As colunas loan_id, coapplicantincome, loan_amount_term, credit_history e property_area são desnécessarias, não vão afetar o modelo.

#### Chekando o balanceamento da coluna *target*

In [174]:
count_target = df.Loan_Status.value_counts()
count_target

Loan_Status
Y    422
N    192
Name: count, dtype: int64

In [176]:
print(f'Existe {round(count_target[0]/len(df), 2)}% de chance de acerto em caso de chute.')

Existe 0.69% de chance de acerto em caso de chute.


C:\Users\gusta\AppData\Local\Temp\ipykernel_3584\2699876029.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'Existe {round(count_target[0]/len(df), 2)}% de chance de acerto em caso de chute.')


A base *target* está desbalanceada. Entendemos que **N**(Não liberar espréstimo) é uma classe minoritaria na hiperclasse **Loan_Status**.

In [179]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

## Tratando os dados da base

excluindo colunas não necessarias

In [183]:
df.drop(labels=['Loan_ID', 'CoapplicantIncome', 'Loan_Amount_Term', 'Credit_History', 'Property_Area'], axis=1,inplace=True)

In [185]:
df.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'LoanAmount', 'Loan_Status'],
      dtype='object')

Preenchendo os dados faltantes com a moda

In [188]:
df = df.apply(lambda col: col.fillna(col.mode().iloc[0]) if not col.mode().empty else col)

In [190]:
df.isnull().sum()

Gender             0
Married            0
Dependents         0
Education          0
Self_Employed      0
ApplicantIncome    0
LoanAmount         0
Loan_Status        0
dtype: int64

Não há mais dados faltantes

In [193]:
df['Dependents'] = df['Dependents'].replace('3+', '3')
df['Dependents'] = df['Dependents'].astype(int)

In [195]:
df.Dependents.info()

<class 'pandas.core.series.Series'>
RangeIndex: 614 entries, 0 to 613
Series name: Dependents
Non-Null Count  Dtype
--------------  -----
614 non-null    int32
dtypes: int32(1)
memory usage: 2.5 KB


Agora a feature *Dependets* foi corrigida.

In [251]:
df['Loan_Status'] = df['Loan_Status'].replace('Y', '1')
df['Loan_Status'] = df['Loan_Status'].replace('N', '0')
df['Loan_Status'] = df['Loan_Status'].astype(int)

In [253]:
X = df.drop(columns = 'Loan_Status', axis=1)
y = df.Loan_Status

In [255]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [257]:
colunas_categoricas = X_train.select_dtypes(include="object").columns
colunas_numericas = X_train.select_dtypes(exclude="object").columns
OHE = OneHotEncoder()
scaler = StandardScaler()
model_rfc = RandomForestClassifier()

transformer = ColumnTransformer([
    ('cat_cols', OHE, colunas_categoricas),
    ('num_cols', scaler, colunas_numericas)
])

pipe = Pipeline([("preprocessing", transformer),
                ("classifier", model_rfc)])

pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('cat_cols', OneHotEncoder(),
                                                  Index(['Gender', 'Married', 'Education', 'Self_Employed'], dtype='object')),
                                                 ('num_cols', StandardScaler(),
                                                  Index(['Dependents', 'ApplicantIncome', 'LoanAmount'], dtype='object'))])),
                ('classifier', RandomForestClassifier())])

In [258]:
X_trans = transformer.fit_transform(X_train)
print(X_trans.shape)

(491, 11)


In [261]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

modelos = {
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier(),
    'SGD': SGDClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier()
}

from sklearn.model_selection import cross_val_score

for nome, modelo in modelos.items():
    pipe.steps[-1] = ('classifier', modelo)  # substitui o modelo no pipeline
    scores = cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy')
    print(f'{nome}: média = {scores.mean():.4f}, desvio = {scores.std():.4f}')


RandomForest: média = 0.6069, desvio = 0.0137
GradientBoosting: média = 0.6395, desvio = 0.0243
SGD: média = 0.6171, desvio = 0.0529
DecisionTree: média = 0.6006, desvio = 0.0504
KNN: média = 0.6253, desvio = 0.0294


O melhor modelo foi o SCV.

In [273]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2', None] 
}

grid = GridSearchCV(GradientBoostingClassifier(), param_grid, scoring='accuracy', verbose = 3) #verbose indica a quantidade de detalhamento das mensagens apresentadas no fit
grid.fit(X_train,y_train)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
[CV 1/5] END learning_rate=0.01, max_depth=3, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=nan total time=   0.0s
[CV 2/5] END learning_rate=0.01, max_depth=3, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=nan total time=   0.0s
[CV 3/5] END learning_rate=0.01, max_depth=3, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=nan total time=   0.0s
[CV 4/5] END learning_rate=0.01, max_depth=3, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=nan total time=   0.0s
[CV 5/5] END learning_rate=0.01, max_depth=3, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=nan total time=   0.0s
[CV 1/5] END learning_rate=0.01, max_depth=3, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=nan total time=   0.0s
[CV 2/5] END lear

ValueError: 
All the 1620 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1620 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\gusta\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\gusta\anaconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\gusta\anaconda3\Lib\site-packages\sklearn\ensemble\_gb.py", line 659, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\gusta\anaconda3\Lib\site-packages\sklearn\base.py", line 650, in _validate_data
    X, y = check_X_y(X, y, 

In [ ]:
grid.best_params_

In [242]:
grid.best_estimator_

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'